In [1]:
!pip install dash networkx plotly stable-baselines3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 5.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 6.0 MB/s eta 0:00:0000:01
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Fla

In [3]:
!pip install dash-cytoscape

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 3.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dash-cytoscape: filename=dash_cytoscape-1.0.2-py3-none-any.whl size=4010776 sha256=9fa4d45314d423c37478a117450f42012212aafc349282ab370e1ceb480a5732
  Stored in directory: /Users/katecastillo/Library/Caches/pip/wheels/99/b1/ab/6c999ab288b4849d372e23c0a8f6ece7edb7ffeb8c97959ab0
Successfully built dash-cytoscape

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [20]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_cytoscape as cyto
import networkx as nx
import numpy as np
from stable_baselines3 import PPO
import gymnasium as gym
from dash.dependencies import Input, Output, State

# Create a Retail Network Graph
G = nx.DiGraph()
G.add_edges_from([
    ("Supplier", "Retailer A", {"supply_volume": 5000}),
    ("Supplier", "Retailer B", {"supply_volume": 4000}),
    ("Retailer A", "Product X", {"sales_volume": 300}),
    ("Retailer B", "Product X", {"sales_volume": 250}),
    ("Retailer B", "Product Y", {"sales_volume": 100})
])

# Convert NetworkX graph to Cytoscape format
elements = []
for node in G.nodes:
    elements.append({"data": {"id": node, "label": node}})
for edge in G.edges:
    edge_data = G[edge[0]][edge[1]]
    weight = edge_data.get("supply_volume", edge_data.get("sales_volume", 0))
    elements.append({"data": {"source": edge[0], "target": edge[1], "weight": weight}})

# Define RL Environment
class RetailPricingEnv(gym.Env):
    def __init__(self, initial_supply=5000, initial_sales=300):
        super(RetailPricingEnv, self).__init__()
        self.action_space = gym.spaces.Discrete(3)  # Lower, Maintain, Increase Price
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(2,), dtype=np.float32)
        self.initial_supply = initial_supply
        self.initial_sales = initial_sales
        self.state = np.array([self.initial_supply, self.initial_sales], dtype=np.float32)

    def step(self, action):
        price_factor = [0.9, 1.0, 1.1][action]
        revenue = self.state[0] * price_factor * 10  # Supply Volume * Price * Demand Factor
        self.state = np.array([
            self.initial_supply,  # Keep supply constant
            self.state[1] + np.random.randint(-50, 50)  # Adjust sales volume slightly
        ], dtype=np.float32)
        return self.state, revenue, False, False, {}

    def reset(self, seed=None, options=None):
        self.state = np.array([self.initial_supply, self.initial_sales], dtype=np.float32)
        return self.state, {}

# Initialize Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Retail Network + RL Simulation"),
    
    # Network Graph
    cyto.Cytoscape(
        id='network-graph',
        elements=elements,
        style={'width': '100%', 'height': '400px'},
        layout={'name': 'cose'},
        stylesheet=[{'selector': 'node', 'style': {'content': 'data(label)'}}]
    ),

    # User Inputs
    html.Div([
        html.Label("Initial Supply Volume:"),
        dcc.Input(id="supply-input", type="number", value=5000, step=100, debounce=True),
        
        html.Label("Initial Sales Volume:"),
        dcc.Input(id="sales-input", type="number", value=300, step=10, debounce=True),
        
        html.Label("Training Timesteps:"),
        dcc.Input(id="timesteps-input", type="number", value=5000, step=500, debounce=True),

        html.Button("Run RL Simulation", id="run-rl", n_clicks=0),
        html.Div(id="rl-output")
    ])
])

@app.callback(
    Output("rl-output", "children"),
    [Input("run-rl", "n_clicks")],
    [State("supply-input", "value"),
     State("sales-input", "value"),
     State("timesteps-input", "value")]
)
def run_rl_simulation(n_clicks, supply, sales, timesteps):
    if n_clicks > 0:
        # Create environment with user inputs
        env = RetailPricingEnv(initial_supply=supply, initial_sales=sales)
        model = PPO("MlpPolicy", env, verbose=1)
        model.learn(total_timesteps=timesteps)
        
        state, _ = env.reset()
        action, _ = model.predict(state)
        state, reward, _, _, _ = env.step(action)

        return f"RL Decision: Action {action}, New Sales Volume: {state[1]}, Revenue Impact: PHP{reward:.2f}"
    return ""

if __name__ == '__main__':
    app.run_server(debug=True)
